# Import dependencies

In [3]:
import os
import torch
import torchaudio
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import MFCC

In [4]:
data_path = os.path.dirname(os.getcwd())+"/data"
os.chdir(data_path)
chord_labels = os.listdir(os.path.join(data_path, "Train"))

# Define a custom Dataset in PyTorch

In [8]:
from sklearn.preprocessing import LabelEncoder

class WAVDataset(Dataset):
    def __init__(self, data_dir, audio_sample_rate):
        self.data_dir = data_dir
        self.sampling_rate = audio_sample_rate
        #Tune n_mels, default is 128
        self.transform = MFCC(sample_rate=audio_sample_rate, n_mfcc=13, melkwargs={"n_mels": 64, "n_fft":400})
        
        #Store file paths and target labels
        self.files = []
        self.labels = []
        self.encoded_labels = []

        # Walk through the directories to get audio file paths and labels
        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            if os.path.isdir(label_dir):
                for audio_file in os.listdir(label_dir):
                    if audio_file.endswith('.wav'):
                        audio_file_path = os.path.join(label_dir, audio_file)
                        self.files.append(audio_file_path)
                        self.labels.append(label)

        #One-hot encode string labels as tensors
        le = LabelEncoder()
        self.encoded_labels = le.fit_transform(self.labels)
        self.encoded_labels = torch.from_numpy(self.encoded_labels)
        self.encoded_labels = torch.nn.functional.one_hot(self.encoded_labels)
        

    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        audio_path = self.files[idx]
        label = self.encoded_labels[idx]
        #Extract MFCC data from audio
        waveform, audio_sample_rate = torchaudio.load(audio_path)
        

        #Some files are mono (1 channel), duplicate channel if so
        if waveform.shape[0] == 1:
            waveform = waveform.repeat(2,1)
  
        waveform_MFCC = self.transform(waveform)

        #Input shapes need to be the same, take the average coefficient over time intervals, should output a tensor of shape (2, n_mfcc)
        waveform_MFCC = torch.mean(waveform_MFCC, dim=2)
        
        return waveform_MFCC, label

In [10]:
# Define paths
train_dir = os.path.join(data_path, "Train")
test_dir = os.path.join(data_path, "Test")

# Create instances of custom dataset
train_dataset = WAVDataset(data_dir=train_dir, audio_sample_rate=16000)
test_dataset = WAVDataset(data_dir=test_dir, audio_sample_rate=16000)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
#Set GPU or CPU
device = "cuda" if torch.cuda.is_available() else "cpu"


class ChordClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        
    
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
       

# 4. Create an instance of the model and send it to target device
model_0 = ChordClassifier().to(device)
model_0

IndentationError: expected an indented block (435473637.py, line 16)